In [1]:
import enum

import isort
import sys
import platform
import importlib
from types import ModuleType
import inspect

In [2]:
def get_stdlib_packages():
    if sys.version_info.minor == 10:
        module_names = sys.stdlib_module_names
    else:
        module_names = isort.stdlibs.py38.stdlib

    external_packages = list()
    for name in module_names:
        if name[0] == '_' or name == 'this' or name == 'antigravity':
            continue
        external_packages.append(name)
    return external_packages

def get_real_packages(package_names):
    real_modules = list()
    not_importable_modules = list()
    for name in package_names:
        try:
            importlib.import_module(name)
            real_modules.append(name)
        except:
            not_importable_modules.append(name)
    return real_modules, not_importable_modules


def get_real():
    external_packages = get_stdlib_packages()
    return get_real_packages(external_packages)

In [3]:
def module_dependency(module_names, name):
    if name not in module_names:
        raise Exception(f'{name} is not importable module')
    dp_names = list()

    # try:
    #     importlib.import_module(name)
    # except:
    #     print(f'err name: {name}')
    for key, val in vars(sys.modules[name]).items():
        if isinstance(val, ModuleType):
            md_name = val.__name__

            try:
                index = md_name.index(".")
                md_name = md_name[0:index]
            except:
                pass

            if md_name != name:
                dp_names.append(md_name)
            # print(f'key: {key}, type: {type(val)}, val: {md_name}, module: {md_name}')

    return dp_names

In [4]:
real_modules, _ = get_real()
# print(real_modules)

In [5]:
import pathlib

dp_names = module_dependency(real_modules, 'pathlib')
print(dp_names)

['fnmatch', 'functools', 'io', 'ntpath', 'os', 'posixpath', 're', 'sys']


In [6]:
# import sys
# sys.setrecursionlimit(50000)
#
# def dependency_graph(modules, ad_mtrx, original_modules, count):
#     count = count + 1
#     if count % 10 == 0:
#         print(f'count: {count}')
#     for md in modules:
#         try:
#             dp_names = module_dependency(original_modules, md)
#         except:
#             print(f'{md} is not in stdlib')
#             continue
#
#         for name in dp_names:
#             try:
#                 i = original_modules.index(md)
#                 j = original_modules.index(name)
#                 ad_mtrx[i][j] = 1
#             except:
#                 print(f'md: {md}, name: {name} is not in stdlib')
#                 continue
#
#         dependency_graph(dp_names, ad_mtrx, original_modules, count)
#
# n = len(real_modules)
# ad_mtrx = [[0 for x in range(n)] for y in range(n)]
# dependency_graph(real_modules, ad_mtrx, real_modules, 0)
# print(ad_mtrx)

In [7]:
dp = module_dependency(real_modules, 'os')
print(dp)

['abc', 'sys', 'stat', 'posixpath']


In [8]:
def module_dependency_map(modules):
    md_map = dict()
    for md in modules:
        md_map[md] = module_dependency(modules, md)
    return md_map

md_map = module_dependency_map(real_modules)
print(md_map)

{'pyclbr': ['io', 'sys', 'importlib', 'tokenize'], 'grp': [], 'site': ['sys', 'os', 'builtins', '_sitebuiltins', 'io'], 'asyncore': ['select', 'socket', 'sys', 'time', 'warnings', 'os'], 'copyreg': [], 'cmath': [], 'pydoc': ['builtins', 'importlib', 'inspect', 'io', 'os', 'pkgutil', 'platform', 're', 'sys', 'sysconfig', 'time', 'tokenize', 'urllib', 'warnings'], 'calendar': ['sys', 'datetime', 'locale'], 'gc': [], 'uu': ['binascii', 'os', 'sys'], 'atexit': [], 'mailbox': ['os', 'time', 'calendar', 'socket', 'errno', 'copy', 'warnings', 'email', 'io', 'contextlib', 'fcntl'], 'readline': [], 'runpy': ['sys', 'importlib', 'io', 'types', 'os'], 'unicodedata': [], 'cProfile': ['_lsprof', 'profile'], 'netrc': ['os', 'shlex', 'stat'], 'mmap': [], 'distutils': ['sys'], 'tabnanny': ['os', 'sys', 'tokenize'], 'contextvars': [], 'zipimport': ['importlib', 'importlib', '_imp', 'io', 'marshal', 'sys', 'time'], 'posix': [], 'plistlib': ['binascii', 'codecs', 'contextlib', 'datetime', 'enum', 'iterto

In [9]:
def build_adj_edge_graph(md_map):
    index_list = list()

    for k, v in md_map.items():
        index_list.append(k)

    n = len(index_list)
    adj_list = list()
    for i in range(n):
        adj_list.append([])

    for md in index_list:
        for name in md_map[md]:
            try:
                i = index_list.index(md)
                j = index_list.index(name)
                if j not in adj_list[i]:
                    adj_list[i].append(j)
            except:
                continue

    return adj_list, index_list

In [10]:
adj_list, index_list = build_adj_edge_graph(md_map)
print(adj_list)
# print(index_list)

[[129, 117, 97, 26], [], [117, 180, 144, 129], [31, 139, 117, 192, 35, 180], [], [], [144, 97, 102, 129, 180, 57, 199, 173, 117, 64, 192, 26, 108, 35], [117, 42, 146], [], [122, 180, 117], [], [180, 192, 7, 139, 201, 125, 35, 149, 129, 203, 150], [], [117, 97, 129, 105, 180], [], [114], [180, 40, 138], [], [117], [180, 117, 26], [], [97, 129, 36, 117, 192], [], [122, 169, 203, 42, 69, 55, 180, 173, 109], [129, 180, 109, 117], [117, 180, 201, 99, 192, 139, 3, 60, 88], [88, 55, 173, 117], [88], [102, 83, 103, 180, 6, 117, 38, 192, 26, 124], [129, 180, 35], [10, 144, 12], [], [180, 117, 138], [117, 180, 192, 129, 173, 124, 35, 71, 88, 85, 10], [180, 117], [117], [], [69, 97, 180, 117, 124], [65, 35, 129, 180, 112, 201, 117, 71], [117, 180, 192, 36, 173], [180, 173, 117], [76, 148, 133], [117], [169, 117], [180, 173, 171, 117], [], [44, 180, 57, 117, 13, 38, 74], [171, 65, 129, 32, 180, 59, 173, 117], [171, 103, 180, 116], [], [], [], [180, 35, 173], [], [180, 117, 59, 108], [], [35, 170],

In [11]:
# def build_dependency_graph(md_map):
#     index_list = list()
#
#     for k, v in md_map.items():
#         index_list.append(k)
#
#     n = len(index_list)
#     ad_mtrx = [[0 for x in range(n)] for y in range(n)]
#
#     for md in index_list:
#         for name in md_map[md]:
#             try:
#                 i = index_list.index(md)
#                 j = index_list.index(name)
#                 ad_mtrx[i][j] = 1
#             except:
#                 continue
#
#     return index_list, ad_mtrx
#
#
# index_list, ad_mtrx = build_dependency_graph(md_map)
# # print(f'index list: {index_list}')
# # print(f'ad_mtrx: {ad_mtrx}')
#
# i = index_list.index('locale')
# print(f'dp of locale ---------')
# dp_list = list()
# for j in range(len(index_list)):
#     if ad_mtrx[i][j] == 1:
#         dp_list.append(index_list[j])
# print(dp_list)

In [12]:
dp = module_dependency(real_modules, 'lzma')
print(dp)

# print(index_list.index('builtins')

['builtins', 'io', 'os', '_compression']


In [13]:
def print_cycle(stack, v):
    print(f'stack: {stack}')
    st2 = [stack.pop()]
    while st2[-1] != v:
        st2.append(stack.pop())

    rs = [v]
    while len(st2) > 0:
        # print(st2[-1])
        rs.append(st2[-1])
        stack.append(st2[-1])
        st2.pop()

    print(f'cycle: {rs}')


In [14]:
def process_DFS_tree(graph, stack, visited_vertices):
    for v in graph[stack[-1]]:
        if visited_vertices[v] == 'in_stack':
            print_cycle(stack, v)
        elif visited_vertices[v] == 'not_visited':
            stack.append(v)
            visited_vertices[v] = 'in_stack'
            process_DFS_tree(graph, stack, visited_vertices)

    visited_vertices[stack[-1]] = 'done'
    stack.pop()

In [15]:
def find_cycles(graph):
    n = len(graph)
    visited = list()
    for i in range(n):
        visited.append('not_visited')

    for v in range(n):
        if visited[v] == 'not_visited':
            stack = [v]
            visited[v] = 'in_stack'
            process_DFS_tree(graph, stack, visited)


In [16]:
adj_list, index_list = build_adj_edge_graph(md_map)
find_cycles(adj_list)

stack: [2, 180, 59]
cycle: [180, 180, 59]


In [19]:
print(index_list[180], index_list[59])

# print(index_list[6])
# print(index_list[165])
# print(index_list[53])
# print(index_list[12])
# print(index_list[6])

os posixpath


In [18]:
import mailbox
import os
import posixpath
import xmlrpc

import zipimport
import importlib